<!-- \title{}
\author{}
\maketitle -->
# QISKIT Lab 2 - Quantum States and Operations on 2-qubits

1. S. G. Nana Engo, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
1. J-P. Tchapet Njafa, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
1. P. Djorwe, djorwepp@gmail.com
    * Department of Physics, Faculty of Science, University of Ngaoundere
       
*Adaptation of the __Qiskit Global Summer School 2020.__*

April 2023

*Prérequis cours*
- Chapitre 1 Systèmes Multi-qubits
- Chapitre 2 Portes Quantiques

*Autres documents pertinents (web)*
- [Access IBM Quantum Systems](https://qiskit.org/documentation/install.html#access-ibm-quantum-systems)
- [IBM Quantum Systems Configuration](https://quantum-computing.ibm.com/docs/manage/backends/configuration)
- [Transpile](https://qiskit.org/documentation/apidoc/transpiler.html)
- [IBM Quantum account](https://quantum-computing.ibm.com/docs/manage/account/ibmq)
- [Quantum Circuits](https://qiskit.org/documentation/apidoc/circuit.html)

# États de Bell

Nous pouvons également explorer les portes multi-qubit dans `Qiskit`. Nous allons voir comment créer, à partir de l'état $\ket{000}$, l'état de Bell 
\begin{equation}
|B_{00}\rangle=\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle).
\end{equation}


# Importer les modules nécessaires

In [ ]:
from qiskit import QuantumCircuit 
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.extensions import XGate, CXGate, HGate
from qiskit.visualization import array_to_latex, plot_bloch_multivector, plot_histogram, visualize_transition 

import numpy as np

Qiskit est trop généreusement avec la précision numérique, que nous supprimons pour une meilleure lisibilité de la sortie:

In [ ]:
np.set_printoptions(precision=4, suppress=True)

## Initialiser et visualiser un état un état |00>

Nous allons commencer par visualiser l'état $|00\rangle$.

In [ ]:
sv00 = Statevector.from_label('00')
sv00.draw('latex')

In [ ]:
sv00.draw('qsphere')

La couleur représentant le vecteur $|00\rangle $ que sa phase est $e^{i0}=1$.

In [ ]:
plot_bloch_multivector(sv00, title='2-qubit |00>')

## Construire un circuit d'état de Bell

On utilise une porte Hadamard $\mathtt{H}$ et une porte $\mathtt{CX}$,
\begin{align}
&\mathtt{CX}|yx\rangle =\mathtt{X}^x|y\rangle\otimes|x\rangle,
&&\mathtt{CX}|y0\rangle =\mathtt{X}^0|y\rangle\otimes|x\rangle=|y0\rangle,
&\mathtt{CX}|y1\rangle =\mathtt{X}^1|y\rangle\otimes|x\rangle=|(1-y)1\rangle.
\end{align}
pour créer l'état de Bell
\begin{equation}
|B_{00}\rangle =\mathtt{CX}(\mathbb{I}\otimes\mathtt{H})|00\rangle 
=\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle.
\end{equation}

* Deux opérateurs $\mathtt{A}$ et $\mathtt{B}$ peuvent être combinés en un opérateur de produit tensoriel $\mathtt{A}\otimes \mathtt{B}$ en utilisant les fonctions `Operator.tensor` et `Operator.expand`.

<div class="alert alert-info">
Comme dans Qiskit, le qubit (opérateur) le moins significatif est à l'extrême droite et le plus à l'extrême gauche, c'est-à-dire:
\begin{align*}
& |j_{n-1}...j_1j_0\rangle&&\text{ pour un système de $n$-qubits}\\
&Q_{n-1}\otimes\cdots\otimes Q_1\otimes Q_0&&\text{ pour un produit tensoriel} 
\end{align*}
il faut faire attention aux représentations mathématiques des opérateurs ne sont pas exactement les mêmes que celles des notes de cours.
</div>

Par conséquent, si $\mathtt{A}$ et $\mathtt{B}$ sont des opérateurs à un seul qubit, alors

1. $\mathtt{A}$.tensor ($\mathtt{B}$) = $\mathtt{A}\otimes \mathtt{B}$ aura les sous-systèmes indexés comme matrice $\mathtt{B}$ sur le sous-système 0 et matrice $\mathtt{A}$ sur le sous-système 1;

2. $\mathtt{A}$.expand ($\mathtt{B}$) = $\mathtt{B}\otimes \mathtt{A}$ aura les sous-systèmes indexés en tant que matrice $\mathtt{A}$ sur le sous-système 0 et matrice $\mathtt{B}$ sur le sous-système 1.

### Methode 1 - Statevector

#### Construction de l'opérateur d'évolution

In [ ]:
 H = (Operator(Pauli('X')) + Operator(Pauli('Z')))/np.sqrt(2)

array_to_latex(H.tensor(Pauli('I')), prefix='\mathtt{H}\otimes\mathbb{I}=')

In [ ]:
array_to_latex(H.expand(Pauli('I')), prefix='\mathbb{I}\otimes\mathtt{H}=')

In [ ]:
array_to_latex(Operator(CXGate()), prefix='\mathtt{CX}=')

<div class="alert alert-info">
Cette matrice est cohérente avec le fait que dans Qiskit, $|01\rangle\equiv\underset{\text{qubit 1}}{|0\rangle}\ \underset{\text{qubit 0}}{|1\rangle}$, qui est à l'opposé de ce que nous utilisons dans les notes de cours, $|01\rangle\equiv\underset{\text{qubit 0}}{|0\rangle}\ \underset{\text{qubit 1}}{|1\rangle}$.
</div>

In [ ]:
BellOp = H.expand(Pauli('I'))&Operator(CXGate()) # Note that B & A = A * B
array_to_latex(BellOp, prefix='\mathtt{CX}(\mathbb{I}\otimes\mathtt{H}) = ')

#### Evolution de |00>

In [ ]:
Bell00 = sv00.evolve(BellOp)
Bell00.draw('latex', prefix='|\\text{B}_{00}\\rangle=')

In [ ]:
Bell00.draw('city')

In [ ]:
Bell00.draw('qsphere')

Notez comment cela ressemble beaucoup à une superposition d'un seul qubit avec une phase nulle. Suite à cet __intrication__, il n'est plus possible de traiter les deux qubits individuellement, et ils doivent être considérés comme un seul système.

#### Probabilités

In [ ]:
Bell00.probabilities_dict()

In [ ]:
array_to_latex(Bell00.probabilities())

In [ ]:
Bell00_counts = Bell00.sample_counts(shots=1000)

plot_histogram(Bell00_counts)

Comme nous pouvons le voir ci-dessus, toutes les mesures donnent le résultat `00` ou `11`. En d'autres termes, si le résultat de la mesure pour l'un des qubits est connu, le résultat pour l'autre est entièrement déterminé.

- Probabilités pour mesurer uniquement le qubit-0

In [ ]:
array_to_latex(Bell00.probabilities([0]))

- Probabilités pour mesurer uniquement le qubit-1

In [ ]:
array_to_latex(Bell00.probabilities([1]))

### Exercices

En utilisant une procédure similaire créer respectivement les états
\begin{align*}
&\frac{1}{\sqrt{2}}(|01\rangle + |10\rangle), &\frac{1}{\sqrt{2}}(|01\rangle - |10\rangle).
\end{align*}

#### Methode 2  -  QuantumCircuit


In [ ]:
circuitQ = QuantumCircuit(2)

# Add a H gate on qubit 0, putting this qubit in superposition.
circuitQ.h(0)

# Add a CX (CNOT) gate on control qubit 0 and target qubit 1, putting
# the qubits in a Bell state.
circuitQ.cx(0, 1)

# Draw the circuit with 'mpl'
circuitQ.draw('mpl',initial_state=True) 

In [ ]:
array_to_latex(Operator(circuitQ),prefix='\mathtt{CX}(\mathtt{H}\otimes\mathbb{I})=')

* Le résultat de ce circuit quantique sur l'état $\ket{000}$ est trouvé en écrivant

In [ ]:
Bell00 = sv00.evolve(circuitQ)
Bell00.draw('latex', prefix='|B_{00}\\rangle=')

In [ ]:
Bell00_counts = Bell00.sample_counts(shots=1000)

plot_histogram(Bell00_counts)

* __Exercice__: Écrivez script Qiskit pour exprimer les quatre états de Bell:
\begin{equation}
|B_{yx}\rangle=\frac{1}{\sqrt{2}}(|y0\rangle + (-1)^x|(1-y)1\rangle).
\end{equation}



In [ ]:
def state(X):
    sv = Statevector.from_label(X) # X est une chaine de  2 caractères (bits)
    mycircuit = QuantumCircuit(2)
    mycircuit.h(0)
    mycircuit.cx(0,1)
    
    new_sv = sv.evolve(mycircuit)
    return new_sv

for x in ['00', '01','10','11']:
    new_sv = state(x)
   
    print(f'B{x} = {new_sv.data}\n')

<div class="alert alert-info">
Ces résultats sont cohérents avec le fait que dans Qiskit, $|01\rangle\equiv\underset{\text{qubit 1}}{|0\rangle}\ \underset{\text{qubit 0}}{|1\rangle}$, qui est à l'opposé de ce que nous utilisons dans les notes de cours, $|01\rangle\equiv\underset{\text{qubit 0}}{|0\rangle}\ \underset{\text{qubit 1}}{|1\rangle}$.
</div>

## Quick Exercises

1.	Write down the tensor product of the qubits:    
    a)	$|0\rangle|1\rangle$    
    b)	$|0\rangle|+\rangle$    
    c)	$|+\rangle|1\rangle$    
    d)	$|-\rangle|+\rangle$    

2.	Write the state $|\psi\rangle = \tfrac{1}{\sqrt{2}}|00\rangle + \tfrac{i}{\sqrt{2}}|01\rangle $ as two separate qubits.


# Mesures

Jusqu'à présent, nous avons simulé l'action d'une mesure en comptant les échantillons à partir du `statevector`. Une mesure peut également être explicitement insérée dans un circuit quantique (`QuantumCircuit`). Voici un exemple qui crée le même état de Bell et applique une mesure.


## Construction et visualisation du circuit

In [ ]:
# Create a Quantum Circuit acting on the q register
circuit = QuantumCircuit(2, 2)

# Add a H gate on qubit 0
circuit.h(0)
circuit.barrier()

# Add a CX (CNOT) gate on control qubit 0 and target qubit 1
circuit.cx(0, 1)
circuit.barrier()

# Map the quantum measurement to the classical bits
circuit.measure([0,1], [0,1])
circuit.draw('mpl',initial_state=True)


Les portes sont ajoutées une par une pour former l'état de Bell souhaité.

- `QuantumCircuit.h (qubit _index)`: appliquer la porte de Hadamar $\mathtt{H}$ sur le qubit indexé par `qubit_ index` (ici l'indice est $0$ pour le premier qubit) et le mettre en superposition;
- `QuantumCircuit.cx (int _control, int_ target)`: applique la porte $\mathtt{CNOT}$. `int _control` est l'indice du qubit de contrôle (ici c'est le premier qubit indexé par $0$) tandis que` int_ target` est l'indice du qubit cible (ici le deuxième qubit indexé par $1$). Ici, cette porte met nos qubits dans un état *intriqué*;
- `QuantumCircuit.measure ([Measur _1st_ qb, Measur _2nd_ qb, ...], [cb _Store_ 1st _qb_ résultat , cb _Store_ 2nd _qb_ result, ...]) `: permet de mesurer un qubit et de stocker le résultat dans un bit classique dédié.

Ici, si vous passez la totalité des registres quantique et classique à `measure`, le résultat de la mesure du *i* ème qubit sera stocké dans le *i* ème bit classique.

## Simuler l'expérience

Qiskit Aer est un cadre de simulation haute performance pour les circuits quantiques. Il fournit plusieurs _backends_ pour atteindre différents objectifs de simulation.

Pour simuler notre circuit en mode expérience, nous allons utilisez le simulateur `QasmSimulator`. Chaque exécution de ce circuit donnera la chaîne de bits 00 ou 11.

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import transpile

# Select the QasmSimulator from the Aer provider
q_sim = AerSimulator()

#Compile the circuit (by adding the transpiler) to reduce the circuit to QASM instructions
# supported by the backend
compiled_circuit = transpile(circuit, q_sim)

# Run the simulator on 1000 shots
job = q_sim.run(compiled_circuit, shots=1000)

# Collect results and count
result = job.result()
B00counts = result.get_counts(compiled_circuit)

# Print 
print("\nTotal count for 00 and 11 are:",B00counts)

Comme prévu, la chaîne de bits de sortie est '00' environ $50\%$ du temps. Le nombre d'exécutions du circuit peut être spécifié via l'argument `shots` dont la valeur par défaut est 1024.

Une fois que vous avez un objet `result`, vous pouvez accéder aux décomptes via `get_counts(circuit)`. Cela vous donne les résultats globaux de l'expérience que vous avez menée.

<div class="alert alert-info">
Il est important de savoir que lors de l'exécution d'un circuit sur un véritable ordinateur quantique, les cicruits doivent généralement être transpilés pour le backend que vous sélectionnez afin que le circuit ne contienne que les portes que le claculateur quantique peut réellement exécuter. Cela implique principalement l'ajout de portes d'échange afin que les portes 2-qubits du circuit correspondent aux paires de qubits sur l'appareil qui peuvent réellement exécuter ces portes.
 </div>

## Visualisez les résultats


In [ ]:
plot_histogram(B00counts)

Nous voyons qu'environ $50\%$ de l'époque, la chaîne de bits de sortie est $00$.

# Autres exemples

## Exemple 1

Considérons un état de produit de 2 qubits $|\psi\rangle=|+\rangle\otimes|0\rangle$.

In [ ]:
psi = Statevector.from_label('+0')

1. Probabilités de mesurer les deux qubits

In [ ]:
psi.probabilities()

2. Probabilités de mesurer uniquement le qubit-0

In [ ]:
psi.probabilities([0])

3. Probabilités de mesurer uniquement le qubit-1

In [ ]:
psi.probabilities([1])

## Exemple 2

Nous pouvons utiliser la propriété `statevector.todic ()` pour convertir un statevector en dictionnaire. Pour $|\psi\rangle=|-\rangle\otimes|0\rangle$,

In [ ]:
psi = Statevector.from_label('-0')
psi.to_dict()

# Exercices

## État GHZ

Créer l'état $\mathtt{GHZ}$
\begin{equation}
|\rm{GHZ}\rangle=\mathtt{CX}_{20}\mathtt{CX}_{10}(\mathbb{I}\otimes\mathbb{I}\otimes\mathtt{H})|000\rangle 
=\frac{1}{\sqrt{2}}\left(|000\rangle + |111\rangle\right),
\end{equation}
et évaluer la probabilité de trouver la chaîne de bits $111$.

## Portes Multi Hadamard

Utilisez les portes Hadamard pour construire un circuit qui produit en sortie (cf. vos notes de cours), avec l'état initial $\ket{000}$,
\begin{equation}
\frac{1}{\sqrt{8}}(|000\rangle + |001\rangle + \cdots + |111\rangle).
\end{equation}

## Retropropagation de la phase

1. Créez l'état $|+-\rangle$ avec un `Statevector` et visualisez avec `plot_bloch_multivector ()`.
2. Donnez le résultat de l'action de la porte CX sur celle-ci et visualisez avec `plot_bloch_multivector ()`.
3. Vérifiez que $\mathtt{CX}_{[01]}\equiv\mathtt{(H\otimes H)}\mathtt{CX}_{[10]}\mathtt{(H\otimes H)}$.

In [ ]:
import qiskit.tools.jupyter #only in jupyter notebook
%qiskit_version_table